NameError: name 'np' is not defined

In [ ]:
'''
make_submission() generates predictions for the Kaggle Painter by Numbers competion
using simple features (image size, aspect ratio and bits/pixel^2)
author: Swaroop Krothapalli - extended code of small yello duck
https://github.com/swaroop7/painters
'''
import time
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.cross_validation import KFold
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score  
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from scipy.stats import itemfreq
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
np.set_printoptions(precision=3, linewidth=100)

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

os.chdir('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Pilot\\Artwork_for_Testing')

In [ ]:
x_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Pilot\\Artwork_for_Testing\\x_train_pilot.txt')
y_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Pilot\\Artwork_for_Testing\\y_train_pilot.txt')

#x_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Wyeths\\x_train_wyeth.txt')
#y_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Wyeths\\y_train_wyeth.txt')


x_train = x_train[:,7]  # Index has to be varied for each feature. 
y_train = y_train

print x_train.shape, y_train.shape
training, testing, y_training, y_testing = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)
actual_training, validation, y_actual_training, y_validation = train_test_split(training, y_training, test_size=0.2, stratify=y_training, random_state=42)



In [2]:
def print_results(clf, y_test, y_pred, y_pred_prob):
    #y_pred_prob = clf.predict_proba(y_test)[:,1]
    #y_pred = clf.predict(y_test)
    print 'ROC - ',roc_auc_score(y_test, y_pred_prob)
    #print 'Confusion Matrix - ', confusion_matrix(y_test, y_pred)
    #print 'Precision - ',precision_score(y_test, y_pred ),'Recall - ',recall_score(y_test, y_pred),'F1- Score',f1_score(y_test, y_pred),'\n'
    target_names = ['class 0', 'class 1']
    print classification_report(y_test, y_pred, target_names=target_names)

In [ ]:
def train_classifier(x_train, y_train, x_cv, y_cv):    
    clf1 = SVC(kernel = 'sigmoid', probability=True, class_weight='balanced')
    clf2 = SVC(kernel = 'rbf', probability=True, class_weight='balanced')
    clf3 = RandomForestClassifier(n_estimators=5, class_weight='balanced')
    clf4 = GaussianNB()
    clf5 = BernoulliNB() 
    clf6 = LinearSVC(class_weight = 'balanced', dual = False)
    clf7 = SVC(kernel = 'poly', probability=True, class_weight='balanced', degree=2, C=1.0, tol = 0.1)
    clf8 = neighbors.KNeighborsClassifier()
    clf9 = linear_model.LogisticRegression( class_weight='balanced')
    clf10 = xgb.XGBClassifier(max_depth=2, n_estimators=10, learning_rate=0.05) #objective='multi:softprob'
    print 'starting fit'
    #print x_train

    print "SVM - Sigmoid Scores"
    clf1.fit(x_train.reshape((19,1)), y_train)
    scores = cross_validation.cross_val_score(clf1, x_train[:, None], y_train, cv=5)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    print "validation_score",accuracy_score(y_validation, clf1.predict(x_cv[:, None]))
    print_results(clf1, y_testing, clf1.predict(testing[:, None]), clf1.predict_proba(testing[:, None])[:,1] )    

    print "SVM - RBF Kernel"
    clf2.fit(x_train.reshape((19,1)), y_train)
    scores = cross_validation.cross_val_score(clf1, x_train[:, None], y_train, cv=5)
    print "validation_score",accuracy_score(y_validation, clf2.predict(x_cv[:, None]))
    print_results(clf2, y_testing, clf2.predict(testing[:, None]), clf2.predict_proba(testing[:, None])[:,1] )    
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    print "Random Forest"
    clf3.fit(x_train.reshape((19,1)), y_train)
    scores = cross_validation.cross_val_score(clf3,  x_train[:, None], y_train, cv=5)
    print "validation_score",accuracy_score(y_validation, clf3.predict(x_cv[:, None]))
    print_results(clf3, y_testing, clf3.predict(testing[:, None]), clf3.predict_proba(testing[:, None])[:,1] )    
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    print "Gaussian NB"
    clf4.fit(x_train.reshape((19,1)), y_train)
    scores = cross_validation.cross_val_score(clf4, x_train[:, None], y_train, cv=3)
    print "validation_score",accuracy_score(y_validation, clf4.predict(x_cv[:, None]))
    print_results(clf4, y_testing, clf4.predict(testing[:, None]), clf4.predict_proba(testing[:, None])[:,1] )        
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    print "Bernoulli NB"
    clf5.fit(x_train.reshape((19,1)), y_train) 
    scores = cross_validation.cross_val_score(clf5,  x_train[:, None], y_train, cv=5)
    print "validation_score",accuracy_score(y_validation, clf5.predict(x_cv[:, None]))
    print_results(clf5, y_testing, clf5.predict(testing[:, None]), clf5.predict_proba(testing[:, None])[:,1] )    
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    print "SVM - Linear Kernel"
    clf6.fit(x_train.reshape((19,1)), y_train)
    scores = cross_validation.cross_val_score(clf6, x_train[:, None], y_train, cv=5)
    print "validation_score",accuracy_score(y_validation, clf6.predict(x_cv[:, None]))
    #print_results(clf6, y_testing, clf6.predict(testing[:, None]), clf6.predict_proba(testing[:, None])[:,1] )    
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    #clf7.fit(x_train.reshape((19,1)), y_train) 
    print "KNN"
    clf8.fit(x_train.reshape((19,1)), y_train) 
    scores = cross_validation.cross_val_score(clf8,  x_train[:, None], y_train, cv=5)
    print "validation_score",accuracy_score(y_validation, clf8.predict(x_cv[:, None]))
    print_results(clf8, y_testing, clf8.predict(testing[:, None]), clf8.predict_proba(testing[:, None])[:,1] )    
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    print "Logistic"
    clf9.fit(x_train.reshape((19,1)), y_train) 
    scores = cross_validation.cross_val_score(clf9,  x_train[:, None], y_train, cv=5)
    print "validation_score",accuracy_score(y_validation, clf9.predict(x_cv[:, None]))
    print_results(clf9, y_testing, clf9.predict(testing[:, None]), clf9.predict_proba(testing[:, None])[:,1] )    
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    print "XGBoost"
    clf10.fit(x_train.reshape((19,1)), y_train)
    scores = cross_validation.cross_val_score(clf10,  x_train[:, None], y_train, cv=5)
    print "validation_score",accuracy_score(y_validation, clf10.predict(x_cv[:, None]))
    print_results(clf10, y_testing, clf10.predict(testing[:, None]), clf10.predict_proba(testing[:, None])[:,1] )    
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    
print actual_training.shape
print y_actual_training.shape
print validation.shape
print y_validation.shape
train_classifier(actual_training, y_actual_training, validation, y_validation)
